In [48]:
# Imports
import math
import torch
import matplotlib

import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim


In [78]:
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor,Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)


In [92]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [100]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

    def train_loop(self, dataloader, model, loss_fn, optimizer):
        size = len(dataloader.dataset)
        for batch, (X, y) in enumerate(dataloader):
            # Compute prediction and loss
            pred = model(X)
            loss = loss_fn(pred, y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch % 100 == 0:
                loss, current = loss.item(), batch * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


    def test_loop(self, dataloader, model, loss_fn):
        size = len(dataloader.dataset)
        test_loss, correct = 0, 0

        with torch.no_grad():
            for X, y in dataloader:
                pred = model(X)
                test_loss += loss_fn(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        test_loss /= size
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [101]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


In [102]:
model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [103]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7])


In [104]:
%matplotlib inline
import torch
x = torch.ones(5)
y = torch.zeros(3)
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [105]:
print('Gradient function for z =',z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x000001F300208E80>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x000001F3002090C0>


In [106]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.1043, 0.0718, 0.1687],
        [0.1043, 0.0718, 0.1687],
        [0.1043, 0.0718, 0.1687],
        [0.1043, 0.0718, 0.1687],
        [0.1043, 0.0718, 0.1687]])
tensor([0.1043, 0.0718, 0.1687])


In [107]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


In [108]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


In [109]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [110]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")

    model.train_loop(train_dataloader, model, loss_fn, optimizer)
    model.test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298276  [    0/60000]
loss: 2.292351  [ 6400/60000]
loss: 2.279995  [12800/60000]
loss: 2.285454  [19200/60000]
loss: 2.269860  [25600/60000]
loss: 2.250703  [32000/60000]
loss: 2.268472  [38400/60000]
loss: 2.243649  [44800/60000]
loss: 2.229423  [51200/60000]
loss: 2.227210  [57600/60000]
Test Error: 
 Accuracy: 41.7%, Avg loss: 0.034981 

Epoch 2
-------------------------------
loss: 2.207881  [    0/60000]
loss: 2.208300  [ 6400/60000]
loss: 2.187683  [12800/60000]
loss: 2.228541  [19200/60000]
loss: 2.173509  [25600/60000]
loss: 2.143873  [32000/60000]
loss: 2.187097  [38400/60000]
loss: 2.139836  [44800/60000]
loss: 2.119034  [51200/60000]
loss: 2.114099  [57600/60000]
Test Error: 
 Accuracy: 42.6%, Avg loss: 0.033358 

Epoch 3
-------------------------------
loss: 2.083110  [    0/60000]
loss: 2.083461  [ 6400/60000]
loss: 2.051262  [12800/60000]
loss: 2.141989  [19200/60000]
loss: 2.015906  [25600/60000]
loss: 1.984955  [32000/600

In [111]:
torch.save(model.state_dict(), "data/model.pth")

print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth
